# Binary orbits for the progenitor of J08408

Study of orbital changes induced by momentum kicks applied at core collapse to a binary system resembling J08408

## Configuration

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from scipy.optimize import bisect

# some constants
one_third = 1e0 / 3e0
pi = 3.1415926535897932384626433832795028841971693993751e0
standard_cgrav = 6.67428e-8
Msun = 1.9892e33
Rsun = 6.9598e10
Hubble_time = 13.461701658024014e0  # Gyr
clight = 2.99792458e10
secyer = 3.1558149984e7

# ---------------------------------------------
# Set some global options
np.seterr(all="ignore")

# ---------------------------------------------
# Load data
root = ".."
run_location = f"{root}/data/natal-kicks"
config_location = f"{root}/config"

idfull, Pfull, afull, efull = np.loadtxt(
    f"{run_location}/all_orbits.data", skiprows=1, unpack=True
)
iddist, wdist, thetadist, phidist = np.loadtxt(
    f"{run_location}/kicks-distribution.data", skiprows=1, unpack=True
)
idgrid, Pgrid, agrid, egrid, probgrid = np.loadtxt(
    f"{run_location}/target_orbits.data", skiprows=9, unpack=True
)

mpl_style = f"{config_location}/style.mpl"
paper_style = f"{config_location}/paper-style.mpl"

## Parameters at core collapse

In [ ]:
m1precc = 8.3525627823294126e000
m2precc = 3.2637938645139933e001
aprecc = 7.3604203503769526e001
pprecc = 1.1424984631360623e001
r1precc = 2.4346822488037647e000
r2precc = 1.8891193348004055e001
remnantmass = 1.6601196874643882e000

## Kepler law & some auxiliary functions

In [ ]:
def a_to_P(separation, m1, m2):
    """Kepler law to go from separation to orbital period

    Parameters
    ----------
    a: binary separation in [Rsun]
    m1: mass of primary star in [Msun]
    m2: mass of secondary star in [Msun]

    Returns
    -------
    period: orbital period in [days]
    """
    separation = separation * Rsun  # in cm
    m1 = m1 * Msun
    m2 = m2 * Msun  # in g

    period = (
        2 * np.pi * np.sqrt(separation**3 / (standard_cgrav * (m1 + m2))) / (86400.0)
    )
    return period


def P_to_a(period, m1, m2):
    """
    Binary separation from known period

    Parameters
    ----------
    P: binary period in [days]
    M1: mass of primary star in [Msun]
    M2: mass of secondary star in [Msun]

    Returns
    -------
    a: binary separation in [Rsun]
    """
    period = period * 24e0 * 3600e0  # in sec
    m1 = m1 * Msun
    m2 = m2 * Msun  # in g
    tmp = standard_cgrav * (m1 + m2) * (period / (2 * np.pi)) ** 2
    separation = np.power(tmp, one_third)
    return separation / Rsun


def rlobe(m1, m2, separation):
    """Roche lobe approximation for a star using Eggleton (1983)
    formula

    Parameters
    ----------
    m1: mass of primary star in [Msun]
    m2: mass of secondary star in [Msun]
    separation: binary separation in [Rsun]

    Returns
    -------
    RL: Roche lobe of primary star in [Rsun]
    """
    one_third = 1e0 / 3e0
    q = np.power(m1 / m2, one_third)
    RL = 0.49e0 * q**2 / (0.6 * q**2 + np.log(1 + q))
    RL = separation * RL

    return RL

## Momentum kick (natal kick)

Here we present the assumptions we define for the collapse stage. These are obtained from other works in which
observations were used to derived statistical distributions for the asymmetric collapse stage.

Given that the natal kick is just a velocity imparted to the compact object remnant of the collapse, it needs to
be defined in terms of its strength and direction on the space. We label as $w$ to the strength of the kick while $\theta$ and $\phi$ to directions.

We consider a Maxwellian distribution for $w$ that is isotropically directed in space. In addition to this, we assume that the strength is reduced by the fraction that fallsback to the proto NS

In [ ]:
plt.style.use("../config/style.mpl")
fig = plt.figure(figsize=(7, 5))
gs = gridspec.GridSpec(2, 2)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, 0])
ax1.set_xlabel("$w$ [km s$^{-1}$]")
ax2.set_xlabel("$\\cos\,\\phi$")
ax3.set_xlabel("$\\theta$")
ax1.set_ylabel("number")
ax3.set_ylabel("number")

# number of bins (only for the plot)
nbins = 30

# distribution
ax1.hist(wdist, bins=nbins, histtype="step", color="C0")
ax2.hist(np.cos(thetadist), bins=nbins, histtype="step", color="C0")
ax3.hist(phidist, bins=nbins, histtype="step", color="C0")

ax1.annotate(
    "$\\sigma \\, [{\\rm km}\\,{\\rm s}^{-1}] = 265 \\times (1 - f_{\\rm fb})$",
    xy=(0.27, 0.76),
    xycoords="figure fraction",
)

# plt.tight_layout()
plt.subplots_adjust(wspace=0.25, hspace=0.27, left=0.05, top=0.9, bottom=0.15)

## Orbital parameter distributions after asymmetric core collapse

With the equations of momentum, the derivation of orbital parameters is easily done. This
conservation implies that certain regions of the space are forbbiden, thus no binaries can
be found in those locations

In this region, one can also derived lines of constant values for the kick, as we will
show later

In [ ]:
plt.style.use("../config/style.mpl")
fig, ax = plt.subplots(figsize=(2.7, 2.7))
ax.set_xscale("log")

# axis labels
latex = "{\\rm orb}"
ax.set_xlabel(f"$P_{latex}$ [days]")
ax.set_ylabel("eccentricity")

# axis limits
ax.set_xlim(3, 2e3)
ax.set_ylim(0, 1)

# theoretical limits
ecc_max = 0.999
ecs = np.linspace(0, ecc_max)
ax.plot(
    a_to_P(aprecc / (1 + ecs), m2precc, remnantmass),
    ecs,
    ls=":",
    c="black",
    zorder=99,
    lw=2,
)
ax.plot(
    a_to_P(aprecc / (1 - ecs), m2precc, remnantmass),
    ecs,
    ls=":",
    c="black",
    zorder=99,
    lw=2,
)

# plot actual randomly computed binary
ax.scatter(Pfull, efull, s=0.001, c="gray", zorder=9)

## Interesting features in this diagram

One interesting thing hidden in this plane is the relation with different constant values such as: $w$, radius of
the Roche lobe (RL, [Eggleton 1983](https://ui.adsabs.harvard.edu/abs/1983ApJ...268..368E/abstract))

<div class="alert alert-success">
    <p style="font-size:20px;" class="aligncenter">The radius of the Roche lobe is simply a function of: <strong>mass-ratio</strong> and the <strong>separation</strong>:</p>
    $\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad R_{\rm RL} = \frac{0.49 q^{2/3}}{0.6 q^{2/3} + \ln(1 + q^{1/3})} \times a$
</div>

<div class="alert alert-success">
    <p style="font-size:20px;" class="aligncenter">With the <strong>orbital period</strong> and using <strong>Kepler's law</strong>, the separation is easily obtained</p>
</div>

Here we show them:

In [ ]:
plt.style.use("../config/style.mpl")
fig, ax = plt.subplots(figsize=(2.7, 2.7))
ax.set_xscale("log")

# axis labels
latex = "{\\rm orb}"
ax.set_xlabel(f"$P_{latex}$ [days]")
ax.set_ylabel("eccentricity")

# axis limits
ax.set_xlim(3, 2e3)
ax.set_ylim(0, 1)

# theoretical limits
ecc_max = 0.999
ecs = np.linspace(0, ecc_max)
ax.plot(
    a_to_P(aprecc / (1 + ecs), m2precc, remnantmass),
    ecs,
    ls=":",
    c="black",
    zorder=99,
    lw=2,
)
ax.plot(
    a_to_P(aprecc / (1 - ecs), m2precc, remnantmass),
    ecs,
    ls=":",
    c="black",
    zorder=99,
    lw=2,
)

# constant lines of RLOF
R_div_RL = [0.6, 0.8, 1.0, 1.2]
colors = plt.cm.viridis(np.linspace(0, 1, len(R_div_RL)))
for k, ratio in enumerate(R_div_RL):
    a_for_R_div_RL = np.zeros(len(ecs))
    a = np.logspace(-1, 4)
    for i, e in enumerate(ecs):
        f = lambda a: r2precc / rlobe(m2precc, remnantmass, a * (1 - e)) - ratio
        a_for_R_div_RL[i] = bisect(f, a=0, b=1e6, maxiter=100)

    mask = a_for_R_div_RL > m2precc / (1 + ecs)
    ax.plot(
        a_to_P(a_for_R_div_RL[mask], m2precc, remnantmass),
        ecs[mask],
        c=colors[k],
        ls="--",
        alpha=1,
        zorder=99,
        label="$R_{}/R_{} = {}$".format("{\\rm donor}", "{\\rm RL,peri}", ratio),
    )
leg = ax.legend(
    fancybox=True,
    frameon=True,
    facecolor="lightgray",
    edgecolor="dimgray",
    framealpha=1.0,
    loc="center",
    ncol=1,
    bbox_to_anchor=(0.65, 0.2),
    bbox_transform=ax.transAxes,
)
for line, text in zip(leg.get_lines(), leg.get_texts()):
    text.set_color(line.get_color())

# a = a_pre / (1 + e)
# => e = 1 - (a_pre / a)
porbs = np.logspace(np.log10(np.min(Pfull)), np.log10(np.max(Pfull)))
ecss = np.zeros(len(porbs))
for k, p in enumerate(porbs):
    a = P_to_a(p, m2precc, remnantmass)
    ecss[k] = 1 - aprecc / a

ax.fill_between(porbs, 0, ecss, color="C0")

# a = a_pre / (1 - e)
# => e = -1 + (a_pre / a)
porbs = np.logspace(np.log10(np.min(Pfull)), np.log10(np.max(Pfull)))
ecss = np.zeros(len(porbs))
for k, p in enumerate(porbs):
    a = P_to_a(p, m2precc, remnantmass)
    ecss[k] = -1 + aprecc / a

ax.fill_between(porbs, 0, ecss, color="C0", zorder=99)
ax.fill_between(np.linspace(0, 4.22), 0, 1.01, color="C0", zorder=99)

# theoretical limits
ecc_max = 0.999
ecs = np.linspace(0, ecc_max)
ax.plot(
    a_to_P(aprecc / (1 + ecs), m2precc, remnantmass),
    ecs,
    ls=":",
    c="black",
    zorder=99,
    lw=2,
)
ax.plot(
    a_to_P(aprecc / (1 - ecs), m2precc, remnantmass),
    ecs,
    ls=":",
    c="black",
    zorder=99,
    lw=2,
)

# annotations
ax.annotate("forbidden region", fontsize=8, xy=(80, 0.5), color="white")

ax.annotate(
    "likely CE",
    xy=(8, 0.82),
    zorder=99,
    rotation=30,
    fontsize=8,
    bbox=dict(facecolor="lightgray", edgecolor="dimgray", boxstyle="round, pad=0.4"),
)

# plot actual randomly computed binary
ax.scatter(Pfull, efull, s=0.001, c="gray", zorder=9)

<div class="alert alert-info">
    <p style="font-size:20px;">Forbidden regions (blue) are a consequence of the conservation of linear momentum. More info can be found in Kalogera 1996 and Kalogera 2000</p>
</div>

Using the lines of constant ratio between the non degenerate star to its associated Roche Lobe, we can remove from
our grid exploration all those binaries which, just after the collapse, becomes an interacting binary with matter
being transferred to the NS, as these cases will unavoidably form a common-envelope phase which is hard to remove
given the really high mass of the star

In [ ]:
plt.style.use("../config/style.mpl")
fig, ax = plt.subplots(figsize=(2.7, 2.7))
ax.set_xscale("log")

# axis labels
latex = "{\\rm orb}"
ax.set_xlabel(f"$P_{latex}$ [days]")
ax.set_ylabel("eccentricity")

# axis limits
ax.set_xlim(3, 2e3)
ax.set_ylim(0, 1)

# theoretical limits
ecc_max = 0.999
ecs = np.linspace(0, ecc_max)
ax.plot(
    a_to_P(aprecc / (1 + ecs), m2precc, remnantmass),
    ecs,
    ls=":",
    c="black",
    zorder=99,
    lw=2,
)
ax.plot(
    a_to_P(aprecc / (1 - ecs), m2precc, remnantmass),
    ecs,
    ls=":",
    c="black",
    zorder=99,
    lw=2,
)

# a = a_pre / (1 + e)
# => e = 1 - (a_pre / a)
porbs = np.logspace(np.log10(np.min(Pfull)), np.log10(np.max(Pfull)))
ecss = np.zeros(len(porbs))
for k, p in enumerate(porbs):
    a = P_to_a(p, m2precc, remnantmass)
    ecss[k] = 1 - aprecc / a

ax.fill_between(porbs, 0, ecss, color="C0")

# a = a_pre / (1 - e)
# => e = -1 + (a_pre / a)
porbs = np.logspace(np.log10(np.min(Pfull)), np.log10(np.max(Pfull)))
ecss = np.zeros(len(porbs))
for k, p in enumerate(porbs):
    a = P_to_a(p, m2precc, remnantmass)
    ecss[k] = -1 + aprecc / a

ax.fill_between(porbs, 0, ecss, color="C0", zorder=99)
ax.fill_between(np.linspace(0, 4.22), 0, 1.01, color="C0", zorder=99)

# theoretical limits
ecc_max = 0.999
ecs = np.linspace(0, ecc_max)
ax.plot(
    a_to_P(aprecc / (1 + ecs), m2precc, remnantmass),
    ecs,
    ls=":",
    c="black",
    zorder=99,
    lw=2,
)
ax.plot(
    a_to_P(aprecc / (1 - ecs), m2precc, remnantmass),
    ecs,
    ls=":",
    c="black",
    zorder=99,
    lw=2,
)

# annotations
ax.annotate("forbidden region", fontsize=8, xy=(80, 0.4), color="white")

# plot actual randomly computed binary
ax.scatter(Pgrid, egrid, s=0.1, c="black", zorder=9)